# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
crypto_df=pd.read_csv("crypto_data.csv", index_col=[0])
crypto_df.head()

In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.value_counts(["IsTrading"])

In [ ]:
crypto_df=crypto_df[crypto_df["IsTrading"]==True]

print(crypto_df.shape)
crypto_df.head()

In [ ]:
# Remove the "IsTrading" column. 
crypto_df=crypto_df.drop(['IsTrading'], axis=1)
print(crypto_df.shape)
crypto_df.head()

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df=crypto_df.dropna()
print(crypto_df.shape)
crypto_df.head()

In [ ]:
# Keep the rows where coins are mined.
crypto_df=crypto_df[crypto_df["TotalCoinsMined"]>0]

print(crypto_df.shape)
crypto_df.head()

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coinname_df=crypto_df[['CoinName']]
print(coinname_df.shape)
coinname_df.head()

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df=crypto_df.drop(['CoinName'], axis=1)
print(crypto_df.shape)
crypto_df.head(10)

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
print(X.shape)
X.head(10)

In [ ]:
# Standardize the data with StandardScaler().
X_scaled=StandardScaler().fit_transform(X)

X_scaled


### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca=PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
print(f"PCA Accuracy:\t{pca.explained_variance_ratio_}")

In [ ]:
# Create a DataFrame with the three principal components.
pcs_df= pd.DataFrame(
    data=X_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head(10)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k=list(range(1,11))

for i in k:
    km  = KMeans(n_clusters = i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow =pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title = "Elbow Curve")


Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
model=KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df=pd.concat([crypto_df,pcs_df, coinname_df], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    hover_name = 'CoinName',
    hover_data = ['Algorithm'],
    color='Class',
    symbol='Class',
    width=800,)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'],
                         sortable=True, selectable=True)

In [ ]:
# Print the total number of tradable cryptocurrencies.
print(f"The total number of tradable cryptocurrencies is:\t{len(clustered_df['CoinName'])}")


In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df['TotalCoinSupply']=MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply']])
clustered_df["TotalCoinsMined"]=MinMaxScaler().fit_transform(clustered_df[["TotalCoinsMined"]])

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=clustered_df.loc[:,('TotalCoinSupply', 'TotalCoinsMined')]

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"]=clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"]=clustered_df["Class"]

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    by="Class", 
    hover_cols="CoinName",
    line_color='black',
    size=75
   )
